In [17]:
using Pkg
Pkg.activate("C:/Users/alexa/BattMo.jl/my-env")


  Activating project at `C:\Users\alexa\BattMo.jl\my-env`


In [4]:

using Revise
using BattMo, Jutul
using CSV
using DataFrames
using GLMakie
using Optimisers
using Distributions
using MAT
using Statistics: mean
using Test

include("equilibrium_calibration.jl")
include("function_parameters_MJ1.jl")

[ Info: Loading cell parameters from file: nothing, default set: Chen2020, model template: nothing
[ Info: Loading cell parameters from file: C:\Users\alexa\.julia\dev\BattMo\src\input\defaults\cell_parameters\Chen2020.json, default set: nothing, model template: nothing
Progress   0%|█                                          |  ETA: 2:47:03

Progress   1%|█                                          |  ETA: 0:34:28

Progress   2%|█                                          |  ETA: 0:19:07

Progress   2%|█                                          |  ETA: 0:13:47

Progress   3%|██                                         |  ETA: 0:10:25

Progress   4%|██                                         |  ETA: 0:08:38

Progress   4%|██                                         |  ETA: 0:08:33

Progress   5%|███                                        |  ETA: 0:06:54

Progress   6%|███                                        |  ETA: 0:05:47

Progress   7%|███                                        |  ETA

simpleComputeDiffusionCoefficientElectrolyteDLR (generic function with 1 method)

In [7]:
function get_tV(x)
    t = [state[:Control][:Controller].time for state in x[:states]]
    V = [state[:Control][:Phi][1] for state in x[:states]]
    return (t, V)
end

function get_tV(x::DataFrame)
    return (x[:, 1], x[:, 2])
end

function trapz(x, y)
    sum((x[i+1] - x[i]) * (y[i] + y[i+1]) / 2 for i in 1:length(x)-1)
end

function getProjectDir()
    return dirname(@__DIR__)
end

getProjectDir (generic function with 1 method)

In [ ]:
# Load experimental data

function getExpData(rate="all", flow="discharge")
    if lowercase(flow) == "discharge"
        fn = joinpath(@__DIR__,"MJ1-DLR", "dlroutput.mat")
    else
        error("Only discharge data available")
    end
    data = matread(fn)
    dlroutput = data["dlroutput"]
    num_experiments = size(dlroutput["time"], 2)
    dlrdata = Vector{Dict{String,Any}}(undef, num_experiments)

    for k in 1:num_experiments
        time_h = dlroutput["time"][k]
        time_s = time_h * 3600
        current = dlroutput["current"][k]
        current_segment = Float64.(current[3:end-1])

        dlrdata[k] = Dict(
            "time" => time_s,
            "rawRate" => dlroutput["CRate"][k],
            "E" => dlroutput["voltage"][k],
            "rawI" => -current,
            "I" => abs(mean(current_segment)),
            "cap" => abs(trapz(time_s[3:end-1], current_segment)),
            "DRate" => 1.0 / time_h[end]
        )
    end
    sort!(dlrdata, by=x -> x["DRate"])

    if rate == "low"
        return dlrdata[1]
    elseif rate == "high"
        return dlrdata[end]
    elseif rate == "all"
        return dlrdata
    else
        error("Unknown rate $rate")
    end
end

getExpData (generic function with 3 methods)

In [10]:
exp_data = getExpData("all", "discharge")
@test length(exp_data) >= 3
@test all(haskey.(exp_data, "time"))

Test Passed

In [ ]:
#Sets up the battery model and simulation for the MJ1 cell
function runMJ1()
    cell_parameters = load_cell_parameters(; from_file_path = joinpath(@__DIR__,"mj1_tab1.json"))
    cycling_protocol = load_cycling_protocol(; from_file_path = joinpath(@__DIR__,"custom_discharge2.json"))
    simulation_settings = load_simulation_settings(; from_default_set = "P2D")
    model_settings = load_model_settings(;from_default_set = "P2D") 

    model_setup = LithiumIonBattery(; model_settings)
    sim = Simulation(model_setup, cell_parameters, cycling_protocol; simulation_settings);
    return cycling_protocol, cell_parameters, model_setup, simulation_settings
end

runMJ1 (generic function with 1 method)

In [ ]:
#Calibration of the equilibrium parameters
function equilibriumCalibration(sim)
    t_exp = vec(exp_data[1]["time"])
    V_exp = vec(exp_data[1]["E"])
    I = exp_data[1]["I"]

    vc = VoltageCalibration(t_exp, V_exp, sim)

    free_calibration_parameter!(vc, ["PositiveElectrode","ActiveMaterial","MaximumConcentration"], lower_bound=1e4, upper_bound=1e5)
    free_calibration_parameter!(vc, ["NegativeElectrode","ActiveMaterial","StoichiometricCoefficientAtSOC100"], lower_bound=0.0, upper_bound=1.0)
    free_calibration_parameter!(vc, ["PositiveElectrode","ActiveMaterial","StoichiometricCoefficientAtSOC100"], lower_bound=0.0, upper_bound=1.0)
    free_calibration_parameter!(vc, ["NegativeElectrode","ActiveMaterial","MaximumConcentration"], lower_bound=1e4, upper_bound=1e5)

    x = solve_equilibrium!(vc; I=I)

    set_calibration_parameter!(vc,["NegativeElectrode","ActiveMaterial","StoichiometricCoefficientAtSOC100"], x[3])
    set_calibration_parameter!(vc,["PositiveElectrode","ActiveMaterial","StoichiometricCoefficientAtSOC100"], x[4])
    set_calibration_parameter!(vc,["NegativeElectrode","ActiveMaterial","MaximumConcentration"], x[1])
    set_calibration_parameter!(vc,["PositiveElectrode","ActiveMaterial","MaximumConcentration"], x[2])

    output = get_simulation_input(vc.sim)
    model = output[:model]
    ocp_ne = model[:NeAm].system.params[:ocp_func]
    ocp_pe = model[:PeAm].system.params[:ocp_func]

    Vne = sum(model[:NeAm].domain.representation[:volumes])
    Vpe = sum(model[:PeAm].domain.representation[:volumes])

    eps_ne = model[:NeAm].system.params[:volume_fraction]
    eps_pe = model[:PeAm].system.params[:volume_fraction]

    a_ne = model[:NeAm].system.params[:volume_fractions][1]
    a_pe = model[:PeAm].system.params[:volume_fractions][1]

    F = 96485.33289
    C_exp = exp_data[1]["I"]*exp_data[1]["time"][end]

    Xparam = [Vpe, Vne, a_pe, a_ne, eps_pe, eps_ne]

    mne = vc.sim.cell_parameters["NegativeElectrode"]["ActiveMaterial"]["MaximumConcentration"] * Vne * a_ne * eps_ne
    mpe = vc.sim.cell_parameters["PositiveElectrode"]["ActiveMaterial"]["MaximumConcentration"] * Vpe * a_pe * eps_pe

    θ_100_ne = vc.sim.cell_parameters["NegativeElectrode"]["ActiveMaterial"]["StoichiometricCoefficientAtSOC100"]
    θ_100_pe = vc.sim.cell_parameters["PositiveElectrode"]["ActiveMaterial"]["StoichiometricCoefficientAtSOC100"]
    θ_0_ne = θ_100_ne - C_exp/(F*mne)
    θ_0_pe = θ_100_pe + C_exp/(F*mpe)

    set_calibration_parameter!(vc,["NegativeElectrode","ActiveMaterial","StoichiometricCoefficientAtSOC0"], θ_0_ne)
    set_calibration_parameter!(vc,["PositiveElectrode","ActiveMaterial","StoichiometricCoefficientAtSOC0"], θ_0_pe)

    Veq = compute_equilibrium_voltage(t_exp, x, Xparam, exp_data[1]["I"], ocp_pe, ocp_ne)

    return (vc.sim.cell_parameters, Veq, t_exp)
end

equilibriumCalibration (generic function with 1 method)

In [ ]:
#Calibration of the kinetic parameters
function highRateCalibration(exp_data,cycling_protocol, cell_parameters_calibrated,model_setup,simulation_settings; scaling = :linear)
    t_exp_hr = vec(exp_data[end]["time"])
    V_exp_hr = vec(exp_data[end]["E"])
    I = exp_data[end]["I"]

    cycling_protocol2 = deepcopy(cycling_protocol)
    cycling_protocol2["DRate"] = exp_data[end]["rawRate"]
    sim2 = Simulation(model_setup, cell_parameters_calibrated, cycling_protocol2; simulation_settings)
    output = get_simulation_input(sim2)
    model2 = output[:model]
    sim2.cycling_protocol["DRate"] = I * 3600 / computeCellCapacity(model2)  

    vc2 = VoltageCalibration(t_exp_hr, V_exp_hr, sim2)

    free_calibration_parameter!(vc2, ["NegativeElectrode","ActiveMaterial", "VolumetricSurfaceArea"], lower_bound = 1e3, upper_bound = 1e6)
    free_calibration_parameter!(vc2, ["PositiveElectrode","ActiveMaterial", "VolumetricSurfaceArea"], lower_bound = 1e3, upper_bound = 1e6)
    free_calibration_parameter!(vc2, ["Separator", "BruggemanCoefficient"], lower_bound = 1e-3, upper_bound = 1e1)
    free_calibration_parameter!(vc2, ["NegativeElectrode","ElectrodeCoating", "BruggemanCoefficient"], lower_bound = 1e-3, upper_bound = 1e1)
    free_calibration_parameter!(vc2, ["PositiveElectrode","ElectrodeCoating", "BruggemanCoefficient"], lower_bound = 1e-3, upper_bound = 1e1)
    free_calibration_parameter!(vc2, ["NegativeElectrode","ActiveMaterial", "DiffusionCoefficient"], lower_bound = 1e-16, upper_bound = 1e-10)
    free_calibration_parameter!(vc2, ["PositiveElectrode","ActiveMaterial", "DiffusionCoefficient"], lower_bound = 1e-16, upper_bound = 1e-10)

    print_calibration_overview(vc2)
    cell_parameters_calibrated2, history = solve(vc2;scaling = scaling);

    return cell_parameters_calibrated2,history
end

highRateCalibration (generic function with 1 method)

In [14]:
cycling_protocol,cell_parameters,model_setup, simulation_settings = runMJ1()
sim = Simulation(model_setup, cell_parameters, cycling_protocol; simulation_settings)
cell_parameters_calibrated, V_eq, t_eq = equilibriumCalibration(sim)
cell_parameters_calibrated2,history  = highRateCalibration(exp_data,cycling_protocol,cell_parameters_calibrated,model_setup,simulation_settings; scaling = :log)

CRates = [exp_data[i]["rawRate"] for i in 1:length(exp_data)]
outputs_base = []
outputs_calibrated = []

for i in 1:length(exp_data)
    I = exp_data[i]["I"]
    simuc = Simulation(model_setup, cell_parameters, cycling_protocol; simulation_settings)
    output = get_simulation_input(simuc)
    model = output[:model]
    simuc.cycling_protocol["DRate"] = I * 3600 / computeCellCapacity(model) 

    output = solve(simuc, info_level = -1; accept_invalid = true)
    push!(outputs_base, (I = I, output = output))

    simc = Simulation(model_setup, cell_parameters_calibrated2, cycling_protocol; simulation_settings)
    outputc = get_simulation_input(simc)
    modelc = outputc[:model]
    simc.cycling_protocol["DRate"] =  I * 3600 / computeCellCapacity(modelc)
    output_c = solve(simc, info_level = -1;accept_invalid = true)
    push!(outputs_calibrated, (I = I, output = output_c))
end

✔️ Validation of ModelSettings passed: No issues found.
──────────────────────────────────────────────────


┌ Info: Loading cell parameters from file: c:\Users\alexa\BattMo.jl\calibration_test\mj1_tab1.json, default set: nothing, model template: nothing
└ @ BattMo C:\Users\alexa\.julia\dev\BattMo\src\input\loader.jl:52


🔍 Validation of CellParameters failed with 1 issue:

──────────────────────────────────────────────────
──────────────────────────────────────────────────
Issue 1:
📍 Where:       [Cell][ElectrodeLength]
🔢 Provided:    1.245
🔑 Rule:        maximum = 0.5
🛠  Issue:       Value exceeds maximum allowed (0.5)

──────────────────────────────────────────────────
✔️ Validation of CyclingProtocol passed: No issues found.
──────────────────────────────────────────────────
✔️ Validation of SimulationSettings passed: No issues found.
──────────────────────────────────────────────────
🔍 Validation of CellParameters failed with 1 issue:

──────────────────────────────────────────────────
──────────────────────────────────────────────────
Issue 1:
📍 Where:       [Cell][ElectrodeLength]
🔢 Provided:    1.245
🔑 Rule:        maximum = 0.5
🛠  Issue:       Value exceeds maximum allowed (0.5)

──────────────────────────────────────────────────
✔️ Validation of CyclingProtocol passed: No issues found.
───────

┌ Info: log scaled BFGS optimization started.
└ @ BattMo C:\Users\alexa\.julia\dev\BattMo\src\calibration\calibration.jl:264


It.  | Objective  | Proj. grad | Linesearch-its
-----------------------------------------------
   0 | 1.7722e+02 | 5.1241e+02 | -
   1 | 1.3647e+02 | 4.0073e+02 | 1
   2 | 5.6862e+01 | 3.4861e+02 | 3
   3 | 5.3841e+01 | 3.9671e+02 | 2
   4 | 4.5344e+01 | 5.3704e+01 | 4
   5 | 3.9034e+01 | 1.7687e+02 | 1
   6 | 1.7138e+01 | 8.2133e+01 | 1
   7 | 4.5104e+00 | 6.1644e+01 | 1
LBFGS: Line search unable to succeed in 5 iterations ...
   8 | 4.5102e+00 | 8.6796e+01 | 5
   9 | 3.7237e+00 | 8.6903e+01 | 2
LBFGS: Line search unable to succeed in 5 iterations ...
  10 | 3.6012e+00 | 6.5699e+01 | 5
LBFGS: Line search unable to succeed in 5 iterations ...
LBFGS: Hessian not updated during iteration 11
  11 | 3.6012e+00 | 6.1354e+01 | 5
Calibration: Calibration finished in 134.9334992 seconds.
🔍 Validation of CellParameters failed with 1 issue:

──────────────────────────────────────────────────
──────────────────────────────────────────────────
Issue 1:
📍 Where:       [Cell][ElectrodeLength]
🔢 Pro

In [15]:
colors = Makie.wong_colors()
for (i, CRate) in enumerate(CRates)
    fig = Figure(size = (800, 500))
    ax = Axis(fig[1, 1], ylabel = "Voltage / V", xlabel = "Time / s", title = "Discharge curve at $(round(CRate, digits = 2))C")

    if i == 1
        lines!(ax, t_eq, V_eq, linestyle = :dash, label = "Equilibrium", color = colors[4])
    end

    t_exp = vec(exp_data[i]["time"])
    V_exp = vec(exp_data[i]["E"])
    lines!(ax, t_exp, V_exp, linestyle = :dot, label = "Experimental", color = colors[1])

    t_sim, V_sim = get_tV(outputs_calibrated[i].output)
    lines!(ax, t_sim, V_sim, linestyle = :dash, label = "Simulated (calibrated)", color = colors[2])

    t_b, V_b = get_tV(outputs_base[i].output)
    lines!(ax, t_b, V_b, linestyle = :dash, label = "Simulated (base)", color = colors[3])

    axislegend(ax, position = :rb)
    display(fig)
    save(joinpath(@__DIR__, "discharge_curve_$(round(CRate, digits=2))C.png"), fig)
end